# Integrantes

- Vicente Rosales
- Nicolas Fuentes

## Codigo

In [77]:
import numpy as np
import pandas as pd

In [78]:
# ---- Header ---- #

# def generar_items(n_items, n_dims)

# def evaluar_solucion(sol_bin, valores, pesos, capacidades)

# def sigmoid(x)

# def sho_binario(valores, pesos, capacidades, n_hienas=30, max_iter=100)

# def leer_instancia_mkp(archivo) 

# ---- Header ---- #

def leer_instancia_mkp(archivo):
    """
    Lee una instancia de MKP desde un archivo en formato OR-Library.
    Retorna: n_items, n_dims, valores, pesos (n_items x n_dims), capacidades
    """
    with open(archivo, 'r') as f:
        lineas = f.readlines()

    idx = 0
    bloques = int(lineas[idx])  # bloques de ítems (por si se usa)
    idx += 1

    n_items, n_dims, _ = map(int, lineas[idx].split())
    idx += 1

    # Leer valores de los ítems
    valores = []
    while len(valores) < n_items:
        valores.extend(map(int, lineas[idx].split()))
        idx += 1

    # Leer pesos por dimensión
    pesos = []
    for d in range(n_dims):
        pesos_dim = []
        while len(pesos_dim) < n_items:
            pesos_dim.extend(map(int, lineas[idx].split()))
            idx += 1
        pesos.append(pesos_dim)
    pesos = np.array(pesos).T  # convertir a (n_items x n_dims)

    # Leer capacidades
    capacidades = []
    while len(capacidades) < n_dims:
        capacidades.extend(map(int, lineas[idx].split()))
        idx += 1

    return n_items, n_dims, np.array(valores), pesos, np.array(capacidades)

def generar_items(n_items, n_dims):
    """
    Throw random shet al problema
    - n_items: número de ítems
    - n_dims: número de dimensiones (restricciones)
    Retorna valores, pesos y capacidades
    """
    # Valores aleatorios entre 1 y 100
    valores = np.random.randint(1, 101, size=n_items)
    # Pesos aleatorios entre 1 y 50 para cada dimensión
    pesos = np.random.randint(1, 51, size=(n_items, n_dims))
    # Cap: para cada dimensión -> una fracción de la suma de pesos
    capacidades = np.zeros(n_dims, dtype=int)
    for d in range(n_dims):
        total_peso = pesos[:, d].sum()
        # Capacidad aleatoria [40% a 60%] de total de pesos
        capacidades[d] = np.random.randint(int(0.4 * total_peso), int(0.6 * total_peso) + 1)
    return valores, pesos, capacidades

def evaluar_solucion(sol_bin, valores, pesos, capacidades):
    """
    Evalúa una solución binaria (vector 0/1) para el MKP
    Calcula el valor total de los ítems seleccionados y aplica lo de la penalización
    proporcional al exceso de peso en cada dimensión
    Retorna (fitness, es_factible)
    """
    total_valor = np.dot(sol_bin, valores)  # suma de valores seleccionados
    # Calcular peso total por dimensión
    peso_total = pesos.T.dot(sol_bin)  # vector de tamaño n_dims
    exceso = np.maximum(0, peso_total - capacidades)  # exceso en cada dimensión
    # Penalización proporcional: factor penal_c * exceso
    penal_c = 10  # factor penalización
    penalizacion = penal_c * exceso.sum()
    fitness = total_valor - penalizacion
    es_factible = np.all(exceso <= 0)
    return fitness, es_factible

def sigmoide(x):
    """Función epica para suavizar para valores."""
    return 1.0 / (1.0 + np.exp(-x))

def sho_binario(valores, pesos, capacidades, n_hienas=30, max_iter=100):
    """
    Esto es SHO para MKP PERO ADAPTADO PARA GENERAR SOLUCIONES BINARIAS!!!!.
    - valores: arreglo de valores de cada ítem (len = n_items)
    - pesos: matriz de pesos (n_items x n_dims)
    - capacidades: vector de capacidades (n_dims)
    - n_hienas: tamaño de población
    - max_iter: número de iteraciones
    Retornar la mejor solución binaria encontrada y su información.
    """
    n_items = len(valores)
    n_dims = len(capacidades)
    # Inicializar posiciones continuas aleatorias en [-1,1]
    poblacion_cont = np.random.uniform(-1, 1, size=(n_hienas, n_items))
    # Variables para mejor solución
    mejor_bin = np.zeros(n_items, dtype=int)
    mejor_valor = -np.inf
    mejor_factible = False
    mejor_cont = None  # Mejor sol (continuo)
    valor_inicial = evaluar_solucion(mejor_bin, valores, pesos, capacidades)[0]

    for it in range(max_iter):
        # 1. Evaluar población actual
        for i in range(n_hienas):
            cont = poblacion_cont[i]
            # Suavizar con sigmoid()
            prob = sigmoide(cont)
            sol_bin = (np.random.rand(n_items) < prob).astype(int)
            fitness, factible = evaluar_solucion(sol_bin, valores, pesos, capacidades)
            # Actualizar mejor global
            if fitness > mejor_valor:
                mejor_valor = fitness
                mejor_bin = sol_bin.copy()
                mejor_factible = factible
                mejor_cont = cont.copy()
        # 2. Actualizar posiciones continuas (Con las fokins ecuaciones de SHO)
        # Parámetro m que decrece linealmente (de 5 a 0) ¡Con esto terminas aumentando o disminuyendo la exploración o explotación!
        m = 5.0 - it * (5.0 / max_iter)
        for i in range(n_hienas):
            if mejor_cont is None:
                continue
            X = poblacion_cont[i]
            # Vectores de coeficientes aleatorios
            B = 2.0 * np.random.rand(n_items)
            E = 2.0 * m * np.random.rand(n_items) - m
            D = np.abs(B * mejor_cont - X)
            # Nueva posición (encircling prey)
            poblacion_cont[i] = mejor_cont - E * D

    return mejor_bin, mejor_valor, mejor_factible, valor_inicial


In [79]:
# Parámetros
np.random.seed(42)  # Fijar semilla para reproducibilidad
n_items = 10       # Número de ítems
n_dims = 3         # Número de restricciones (en el fondo son las dimensiones)
n_hienas = 40      # Tamaño de la población
max_iter = 100     # Número de iteraciones

# Leer instancia fácil
n_items_faciles, n_dims_faciles, valores_faciles, pesos_faciles, capacidades_faciles, valor_inicial_faciles = leer_instancia_mkp("Instancias/mknapcb1.txt")

# Leer instancia media
n_items_dificiles, n_dims_media, valores_media, pesos_media, capacidades_media, valor_inicial_media = leer_instancia_mkp("Instancias/mknapcb5.txt")

# Leer instancia difícil
n_items_dificiles, n_dims_dificiles, valores_dificiles, pesos_dificiles, capacidades_dificiles, valor_inicial_dificiles = leer_instancia_mkp("Instancias/mknapcb9.txt")

ValueError: not enough values to unpack (expected 6, got 5)

## Pruebas

### Instancia facil

In [ ]:
# ejecutar la custion
mejor_bin, mejor_valor, mejor_factible = sho_binario(valores_faciles, pesos_faciles, capacidades_faciles,
                                                    n_hienas=n_hienas, max_iter=max_iter)

# Imprimir resultados
print("Mejor solución encontrada:")
print("Vector binario:\n", mejor_bin.reshape(10, 10))
print("Valor total:", np.dot(mejor_bin, valores_faciles))
print("Factible:", mejor_factible)


Mejor solución encontrada:
Vector binario:
 [[0 0 0 1 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 1 0]
 [1 1 0 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 0]
 [0 0 1 0 1 1 0 0 0 1]
 [1 0 1 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 1 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0]]
Valor total: 18478
Factible: True


/tmp/ipykernel_11821/1737752903.py:93: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


### Instancia media

In [ ]:
# ejecutar la custion
mejor_bin, mejor_valor, mejor_factible = sho_binario(valores_media, pesos_media, capacidades_media,
                                                    n_hienas=n_hienas, max_iter=max_iter)

# Imprimir resultados
print("Mejor solución encontrada:")
print("Vector binario:\n", mejor_bin.reshape(10, 25))
print("Valor total:", np.dot(mejor_bin, valores_media))
print("Factible:", mejor_factible)


Mejor solución encontrada:
Vector binario:
 [[0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1]
 [1 0 0 0 0 0 0 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 1 1 1]
 [1 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 1]
 [1 0 1 0 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0]
 [0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 1 0]
 [0 1 1 1 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0]]
Valor total: 55873
Factible: False


/tmp/ipykernel_11821/1737752903.py:93: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


### Instancia dificil

In [ ]:
# ejecutar la custion
mejor_bin, mejor_valor, mejor_factible = sho_binario(valores_dificiles, pesos_dificiles, capacidades_dificiles,
                                                    n_hienas=n_hienas, max_iter=max_iter)

# Imprimir resultados
print("Mejor solución encontrada:")
print("Vector binario:\n", mejor_bin.reshape(20,25))
print("Valor total:", np.dot(mejor_bin, valores_dificiles))
print("Factible:", mejor_factible)


/tmp/ipykernel_11821/1737752903.py:93: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


Mejor solución encontrada:
Vector binario:
 [[0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0 0 0 1]
 [1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]
 [1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1]
 [1 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 1]
 [1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1]
 [1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0]
 [1 1 0 0 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1]
 [0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 1 1 0 1 1 1 0 1]
 [0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 0 0 1 0]
 [1 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 1 0 1 1 1 1 0 1 0]
 [1 0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1]
 [

## Analisis

In [ ]:
instancias_txt = {
    'Fácil': 'Instancias/mknapcb1.txt',
    'Media': 'Instancias/mknapcb1.txt',
    'Difícil': 'Instancias/mknapcb1.txt',
}

n_runs = 10
results = {nombre: [] for nombre in instancias_txt}
feasibility = {nombre: [] for nombre in instancias_txt}

for nombre, archivo in instancias_txt.items():
    n_items, n_dims, valores, pesos, capacidades = leer_instancia_mkp(archivo)
    for _ in range(n_runs):
        mejor_bin, mejor_valor, factible = sho_binario(valores, pesos, capacidades, n_hienas=30, max_iter=100)
        results[nombre].append(mejor_valor)
        feasibility[nombre].append(int(factible))

df_values = pd.DataFrame(results)
#df_feasibility = pd.DataFrame(feasibility)
df_values.loc['Mean'] = df_values.mean()
df_values.loc['Std'] = df_values.std()

/tmp/ipykernel_11821/1737752903.py:93: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [ ]:
df_values

,Fácil,Media,Difícil
0,20359.000000,19107.000000,18964.000000
1,18485.000000,19194.000000,18969.000000
2,19591.000000,19560.000000,5253.000000
3,19061.000000,18790.000000,19151.000000
4,18960.000000,20120.000000,20945.000000
5,18953.000000,20662.000000,19608.000000
6,17661.000000,18023.000000,20610.000000
7,19986.000000,18572.000000,18124.000000
8,20676.000000,19429.000000,19433.000000
9,20930.000000,-14718.000000,18487.000000


,Fácil,Media,Difícil
0,1,1,1
1,1,1,1
2,1,1,0
3,1,0,1
4,1,1,0
5,1,1,1
6,1,0,0
7,1,1,1
8,1,1,1
9,1,0,1
